In [1]:
from os import makedirs
folder='V:\\V_Drive\\DATA\\train/'
dataset_home = 'dataset_dogs_vs_cats/'
subdirs = ['train/', 'test/']
for subdir in subdirs:
	# create label subdirectories
	labeldirs = ['dogs/', 'cats/']
	for labldir in labeldirs:
		newdir = folder+dataset_home + subdir + labldir
		makedirs(newdir, exist_ok=True)

In [2]:
import shutil as s
import os
import random as r
r.seed(1)
# define ratio of pictures to use for validation
val_ratio = 0.25
# copy training dataset images into subdirectories
src_directory = folder+'train/'
for file in os.listdir(src_directory):
	src = src_directory + '/' + file
	dst_dir = 'train/'
	if r.random() < val_ratio:
		dst_dir = 'test/'
	if file.startswith('cat'):
		dst = folder+'dataset_dogs_vs_cats/'+ dst_dir + 'cats/'  + file
		s.copyfile(src, dst)
	elif file.startswith('dog'):
		dst = folder +'dataset_dogs_vs_cats/'+ dst_dir + 'dogs/'  + file
		s.copyfile(src, dst)

In [6]:
from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(rescale=1.0/255.0)
train_it=gen.flow_from_directory('V:\\V_Drive\\DATA\\train\\dataset_dogs_vs_cats/train/',
		class_mode='binary', batch_size=64, target_size=(200, 200))
test_it = gen.flow_from_directory('V:\\V_Drive\\DATA\\train\\dataset_dogs_vs_cats/test/',
		class_mode='binary', batch_size=64, target_size=(200, 200))

Found 18697 images belonging to 2 classes.
Found 6303 images belonging to 2 classes.


In [9]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

In [10]:
model = define_model()

In [11]:
	history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
		validation_data=test_it, validation_steps=len(test_it), epochs=20, verbose=0)

	_, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
	print('> %.3f' % (acc * 100.0))

> 73.600


In [12]:
model.save('final_model.h5')


In [14]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
img=load_img('V:\\V_Drive\\DATA\\train/sample_image.jpg',target_size=(200, 200))
img=img_to_array(img)
img=img.reshape(1, 200, 200,3)
img = img.astype('float32')
img = img - [123.68, 116.779, 103.939]



In [15]:
result = model.predict(img)
if result[0]==0:
    print("Cat")
else:
    print("Dog")

Dog


In [20]:
from matplotlib import pyplot
pyplot.subplot(211)
pyplot.title('Cross Entropy Loss')
pyplot.plot(history.history['loss'], color='blue', label='train')
pyplot.plot(history.history['val_loss'], color='orange', label='test')
pyplot.subplot(212)
pyplot.title('Classification Accuracy')
pyplot.plot(history.history['accuracy'], color='blue', label='train')
pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
filename = 'V:\\V_Drive\\DATA\\train/dog_cat'
pyplot.savefig(filename + '_plot.png')
pyplot.close()